In [245]:
## Set up dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb

df = pd.read_csv("base_dataset.csv")

OIEH#HFNDdjj


In [241]:
## Clean up data

df = df[df['viewCount'].notna()]
# df = df[df['likeCount'].notna()]
# df = df[df['commentCount'].notna()]

def process_title(title):
    title = str(title)
    # title = re.sub(r"&.*?;", " ", title)
    title = re.sub(r"[!\"#\＄%&\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]", " ", title)
    title = title.lower()
    title = title.split(" ")
    title = list(filter(None, title))
    return title

titles = df["vidTitle"].map(process_title)

def process_tags(tags):
    if(tags == None) return []
    tags = str(tags)
    tags = tags.lower()
    tags = tags[1:-1]
    tags = tags.split(,)
    tags = [tag[1:-1] for tag in tags]
    tags = list(filter(None, tags))
    return tags

tags = df["tags"].map(process_tags)
print(tags)
df["tagCount"] = len(tags)

In [242]:
## Train Model for Word2Vec on Titles

from gensim.models import Word2Vec, FastText
import re

vector_size = 300
w2v = Word2Vec(titles, min_count=2, vector_size = vector_size)

def convertToVec(wordlist):
    if (len(title) == 0):
        return np.zeros(vector_size)
    else:
        return np.mean([w2v.wv[word] if (word in w2v.wv) else np.zeros(vector_size) for word in wordlist], axis=0)

test_titles = df.loc[:, "vidTitle"].map(process_title)
title_vectors = [convertToVec(title) for title in test_titles]

title_vec_cols = ["titleVec" + str(num) for num in range(vector_size)]
title_vec_df = pd.DataFrame(title_vectors, index=df.index, columns = title_vec_cols)

df = pd.concat([df, title_vec_df], axis=1)

In [243]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

x_labels = ["categoryId", "tagCount"] + title_vec_cols
y_labels = ["viewCount"]
x = df.loc[:, x_labels]
y = df.loc[:, y_labels]
dmatrix = xgb.DMatrix(data=x,label=y)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror',
                           colsample_bytree = 0.3, 
                           learning_rate = 0.1,
                           max_depth = 10, 
                           alpha = 50, 
                           n_estimators = 10)

xgb_reg.fit(x_train, y_train)
pred = xgb_reg.predict(x_test)

print("MAE: " + str(mean_absolute_error(y_test, pred)))
print("RMSE: " + str(np.sqrt(mean_squared_error(y_test, pred))))

MAE: 34905736.27445652
RMSE: 108904896.02103995


In [7]:
params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.3,
                'max_depth': 10, 'alpha': 10}

cv_results = xgb.cv(dtrain=dmatrix, params=params, nfold=3,
                    num_boost_round=100,early_stopping_rounds=20,metrics="rmse", as_pandas=True, seed=123)

cv_results.describe()
print(cv_results)

   train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
0        91.090988        3.732703       91.093904       8.681391
1        88.495440        3.338890       88.832230       8.356700
2        87.147969        3.127699       87.795114       7.890722
3        86.452434        3.021541       87.354288       7.429542
4        86.097054        2.965904       87.193656       7.037225
5        85.914080        2.936251       87.152253       6.720735
